In [20]:
import numpy as np
from svgpathtools import svg2paths
from PIL import Image, ImageDraw
import heapq

def load_svg_image(filepath, size=(40, 40)):
    # Load SVG paths
    paths, _ = svg2paths(filepath)
    
    # Create a blank white image
    img = Image.new('L', size, 255)
    draw = ImageDraw.Draw(img)

    # Draw each path on the image
    for path in paths:
        for element in path:
            if element.length() > 0:
                points = [(pt.real, pt.imag) for pt in element]
                draw.line(points, fill=0)  # Draw path with black color

    # Convert to numpy array
    image_array = np.array(img)
    return image_array

# Load the generated occupancy grid SVG image
image_path = 'maze.svg'
image_array = load_svg_image(image_path)

# Convert image array to grid format
grid = (image_array == 255).astype(int).tolist()  # Convert white (255) to 1 (free space), black (0) to 0 (obstacle)

# Define start and goal positions
start = (3, 3)
goal = (30, 30)

In [21]:
class Node:
    def __init__(self, parent, position):
        self.parent = parent
        self.position = position
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

    def __lt__(self, other):
        return self.f < other.f

In [22]:
def astar(grid, start, end, visited_nodes):
    """Returns a list of tuples as a path from the given start to the given end in the given grid"""

    # Create start and end node
    start_node = Node(None, start)
    end_node = Node(None, end)

    # Initialize both open and closed lists
    open_list = []
    closed_list = set()

    # Add the start node
    heapq.heappush(open_list, start_node)

    while open_list:
        # Get the current node
        current_node = heapq.heappop(open_list)
        closed_list.add(current_node.position)

        # Found the goal
        if current_node == end_node:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  # Return reversed path

        # Generate children (adjacent squares)
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:  # 4 directions (no diagonals)

            # Get node position
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            # Make sure within range and walkable terrain
            if (0 <= node_position[0] < len(grid) and
                0 <= node_position[1] < len(grid[0]) and
                grid[node_position[0]][node_position[1]] == 1):
                new_node = Node(current_node, node_position)
                children.append(new_node)

        # Loop through children
        for child in children:
            if child.position in closed_list:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = abs(child.position[0] - end_node.position[0]) + abs(child.position[1] - end_node.position[1])
            child.f = child.g + child.h

            # Avoid nodes that have been visited in previous paths but give them lower priority
            if child.position in visited_nodes:
                child.f += 10  # Penalize visited nodes to prioritize unvisited nodes

            # Child is already in the open list
            if any(open_node.position == child.position and child.g > open_node.g for open_node in open_list):
                continue

            # Add the child to the open list
            heapq.heappush(open_list, child)

    return None  # No path found

In [23]:
from tqdm import tqdm

def find_multiple_paths(grid, start, end, num_paths):
    all_paths = []
    visited_nodes = set()

    for _ in range(num_paths):
        path = astar(grid, start, end, visited_nodes)
        print("path found!")
        if path is None:
            break
        all_paths.append(path)
        visited_nodes.update(path)
    
    return all_paths

paths = find_multiple_paths(grid, start, goal, 3)
print(len(paths))
for path in paths:
    print(path)

path found!
0


In [5]:
def draw_routes_on_map(image, routes, output_path):
    image = image.convert("RGB")  # Convert to RGB for coloring
    draw = ImageDraw.Draw(image)
    
    # Draw each route in a different color
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 165, 0)]
    for i, route in enumerate(routes):
        color = colors[i % len(colors)]
        for position in route:
            draw.point(position, fill=color)  # Draw route points in different colors
    
    image.save(output_path)
    
# Load the original image to draw the routes
original_image = Image.open(image_path)
draw_routes_on_map(original_image, paths, 'routes_on_map.png')

In [6]:
import plotly.graph_objects as go

def plot_grid_with_paths_image(grid, paths, start, goal):
    # Normalize the grid values to be in the range [0, 1]
    grid = np.array(grid)
    grid_normalized = (grid - np.min(grid)) / (np.max(grid) - np.min(grid))
    
    # Define a custom colorscale
    colorscale = [
        [0, 'white'],    # Map 0 to white
        [1, 'black']     # Map 1 to black
    ]
    
    # Convert normalized grid values to indices of colorscale
    grid_colored = np.interp(grid_normalized, [0, 1], [0, len(colorscale) - 1])
    
    # Convert grid_colored to an integer type
    grid_colored = grid_colored.astype(int)

    fig = go.Figure()

    # Add the grid as an image
    fig.add_trace(go.Image(
        z=grid_colored,
        zmin=0,
        zmax=len(colorscale) - 1
    ))

    # Add start and goal positions
    fig.add_trace(go.Scatter(
        x=[start[1]], y=[start[0]], 
        mode='markers', marker=dict(color='green', size=15), 
        name='Start'
    ))

    fig.add_trace(go.Scatter(
        x=[goal[1]], y=[goal[0]], 
        mode='markers', marker=dict(color='red', size=15), 
        name='Goal'
    ))

    # Add paths
    for path in paths:
        fig.add_trace(go.Scatter(
            x=[p[1] for p in path], y=[p[0] for p in path],
            mode='lines+markers', line=dict(width=2), 
            name='Path'
        ))

    fig.update_layout(
        title='Grid with Paths',
        xaxis_title='Column Index',
        yaxis_title='Row Index',
        yaxis=dict(autorange='reversed')  # Reverse y-axis to match the grid orientation
    )

    fig.show()

plot_grid_with_paths_image(grid, paths, start, goal)

C:\Users\Izabela\AppData\Local\Temp\ipykernel_14168\650558928.py:6: RuntimeWarning: invalid value encountered in divide
  grid_normalized = (grid - np.min(grid)) / (np.max(grid) - np.min(grid))
C:\Users\Izabela\AppData\Local\Temp\ipykernel_14168\650558928.py:18: RuntimeWarning: invalid value encountered in cast
  grid_colored = grid_colored.astype(int)


ValueError: 
    Invalid value of type 'builtins.int' received for the 'zmax' property of image
        Received value: 1

    The 'zmax' property is an info array that may be specified as:

    * a list or tuple of 4 elements where:
(0) The 'zmax[0]' property is a number and may be specified as:
      - An int or float
(1) The 'zmax[1]' property is a number and may be specified as:
      - An int or float
(2) The 'zmax[2]' property is a number and may be specified as:
      - An int or float
(3) The 'zmax[3]' property is a number and may be specified as:
      - An int or float